In [1]:
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from multielo import MultiElo, Tracker
from mktools.get_data import load_data_pd
from mktools.validate_data import validate_bad_uids
from mktools.form_data import fill_new_session
import plotly.express as px
from alive_progress import alive_it
from bs4 import BeautifulSoup
from typing import Literal

# Load Variables from .env file
load_dotenv()

# LAST_FORM_DATA_DATE = pd.Timestamp('1/27/2025 23:40:20').tz_localize("US/Eastern").tz_convert("UTC")

# LAST_FORM_DATA_DATE

True

In [9]:
migrated_data = pd.read_csv(
    r"C:\Users\Cooper\sandbox\mkstream\form_data_migration\form_data_valid.csv"
)

migrated_data["TIMESTAMP"] = pd.to_datetime(migrated_data["TIMESTAMP"], utc=True).dt.tz_convert("US/Eastern")

In [10]:
pd.Timestamp('2024-07-09 08:40:00').tz_localize("US/Eastern")

Timestamp('2024-07-09 08:40:00-0400', tz='US/Eastern')

In [11]:
pd.Timestamp("2025-01-29 03:58:15").tz_localize("US/Eastern")

Timestamp('2025-01-29 03:58:15-0500', tz='US/Eastern')

In [ ]:
# # Temporarily remove bad data can remove later
# migrated_data = migrated_data[
#     ~migrated_data["TIMESTAMP"].isin(
#         [
#             pd.Timestamp("2024-07-09 08:40:00").tz_localize("US/Eastern"),
#             pd.Timestamp("2024-07-09 08:35:00").tz_localize("US/Eastern"),
#         ]
#     )
# ]

In [12]:
LAST_FORM_DATA_DATE = migrated_data["TIMESTAMP"].max()

LAST_FORM_DATA_DATE

Timestamp('2025-01-28 22:58:15-0500', tz='US/Eastern')

In [13]:
form_df = load_data_pd(sheet_name="form_data", sheet_id=os.environ["SHEET_ID"])

form_df = form_df.drop(
    columns=[
        x
        for x in form_df.columns
        if x.__contains__("Unnamed") or x.__contains__("Score")
    ]
)

form_df["Timestamp"] = pd.to_datetime(form_df["Timestamp"]).dt.tz_localize("US/Eastern")

form_df.columns = [
    x.replace(" ", "_").replace("[", "").replace("]", "") if x.__contains__("[") else x
    for x in form_df.columns
]

In [14]:
new_df = (
    form_df[form_df["Timestamp"] > LAST_FORM_DATA_DATE].copy().reset_index(drop=True)
)

new_df

,Timestamp,NEW_SESSION,MAP,PLAYERS,PLAYERS_2_1ST,PLAYERS_2_2ND,CHARACTERS_2_1ST,CHARACTERS_2_2ND,PLAYERS_3_1ST,PLAYERS_3_2ND,...,CHARACTERS_3_2ND,CHARACTERS_3_3RD,PLAYERS_4_1ST,PLAYERS_4_2ND,PLAYERS_4_3RD,PLAYERS_4_4TH,CHARACTERS_4_1ST,CHARACTERS_4_2ND,CHARACTERS_4_3RD,CHARACTERS_4_4TH
0,2025-01-29 19:13:49-05:00,YES,D.K.'s Jungle,3,NaN,NaN,NaN,NaN,Cooper,Blake,...,Yoshi,Peach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-29 19:13:58-05:00,NO,Koopa Troopa Beach,3,NaN,NaN,NaN,NaN,Matt,Blake,...,Toad,Peach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-29 19:50:03-05:00,NO,Wario Stadium,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Cole,Blake,Matt,Cooper,Toad,Peach,Yoshi,Bowser
3,2025-01-29 20:33:08-05:00,NO,Wario Stadium,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Cole,Connor,Blake,Domingo,Luigi,Yoshi,Peach,Toad
4,2025-01-30 02:03:51-05:00,NO,Sherbet Land,3,NaN,NaN,NaN,NaN,Regan,Connor,...,Peach,Yoshi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2025-02-07 17:30:53-05:00,YES,Yoshi Valley,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Cooper,Cole,Blake,Garrett,Luigi,Toad,Yoshi,Peach
95,2025-02-07 18:20:16-05:00,NO,Sherbet Land,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Blake,Cole,Cooper,Garrett,Luigi,Toad,Yoshi,Peach
96,2025-02-07 18:23:43-05:00,NO,D.K.'s Jungle,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Blake,Cole,Cooper,Garrett,Yoshi,Toad,Mario,Peach
97,2025-02-07 18:29:30-05:00,NO,Koopa Troopa Beach,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Blake,Regan,Cooper,Cole,Peach,Luigi,Yoshi,Toad


In [15]:
if new_df.shape[0] == 0:
    raise IndexError("No new records to update")

In [16]:
def transform_form_data(
    df: pd.DataFrame, game_type_replace_string: Literal["_2_", "_3_", "_4_"]
) -> pd.DataFrame:

    if df.shape[0] > 0:
        idf = df.copy()

        idf_out = idf.dropna(axis=1, how="all").reset_index(drop=True)

        idf_out.columns = [
            (
                x.replace(game_type_replace_string, "_")
                if x.__contains__(game_type_replace_string)
                else x.upper()
            )
            for x in idf_out.columns
        ]

        idf_out["TIMESTAMP"] = pd.to_datetime(idf_out["TIMESTAMP"])

        return idf_out
    else:
        return pd.DataFrame()

In [18]:
dfs = []

for game_type in [2, 3, 4]:
    tdf = new_df[new_df["PLAYERS"] == game_type].copy().reset_index(drop=True)

    dfs.append(tdf)

two_p = dfs[0]
three_p = dfs[1]
four_p = dfs[2]

two_p_out = transform_form_data(df=two_p, game_type_replace_string="_2_")
three_p_out = transform_form_data(df=three_p, game_type_replace_string="_3_")
four_p_out = transform_form_data(df=four_p, game_type_replace_string="_4_")

cat_df = (
    pd.concat([two_p_out, three_p_out, four_p_out])
    .sort_values(by="TIMESTAMP")
    .reset_index(drop=True)
    .reset_index()
)[
    [
        "TIMESTAMP",
        "NEW_SESSION",
        "MAP",
        "PLAYERS",
        "PLAYERS_1ST",
        "PLAYERS_2ND",
        "PLAYERS_3RD",
        "PLAYERS_4TH",
        "CHARACTERS_1ST",
        "CHARACTERS_2ND",
        "CHARACTERS_3RD",
        "CHARACTERS_4TH",
    ]
]

cat_df

,TIMESTAMP,NEW_SESSION,MAP,PLAYERS,PLAYERS_1ST,PLAYERS_2ND,PLAYERS_3RD,PLAYERS_4TH,CHARACTERS_1ST,CHARACTERS_2ND,CHARACTERS_3RD,CHARACTERS_4TH
0,2025-01-29 19:13:49-05:00,YES,D.K.'s Jungle,3,Cooper,Blake,Matt,NaN,Toad,Yoshi,Peach,NaN
1,2025-01-29 19:13:58-05:00,NO,Koopa Troopa Beach,3,Matt,Blake,Cooper,NaN,Yoshi,Toad,Peach,NaN
2,2025-01-29 19:50:03-05:00,NO,Wario Stadium,4,Cole,Blake,Matt,Cooper,Toad,Peach,Yoshi,Bowser
3,2025-01-29 20:33:08-05:00,NO,Wario Stadium,4,Cole,Connor,Blake,Domingo,Luigi,Yoshi,Peach,Toad
4,2025-01-30 02:03:51-05:00,NO,Sherbet Land,3,Regan,Connor,Domingo,NaN,Toad,Peach,Yoshi,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
94,2025-02-07 17:30:53-05:00,YES,Yoshi Valley,4,Cooper,Cole,Blake,Garrett,Luigi,Toad,Yoshi,Peach
95,2025-02-07 18:20:16-05:00,NO,Sherbet Land,4,Blake,Cole,Cooper,Garrett,Luigi,Toad,Yoshi,Peach
96,2025-02-07 18:23:43-05:00,NO,D.K.'s Jungle,4,Blake,Cole,Cooper,Garrett,Yoshi,Toad,Mario,Peach
97,2025-02-07 18:29:30-05:00,NO,Koopa Troopa Beach,4,Blake,Regan,Cooper,Cole,Peach,Luigi,Yoshi,Toad


In [19]:
season_initial = load_data_pd(
    sheet_name="data_main",
    sheet_id=os.environ["SHEET_ID"],
    usecols=[
        "DATE",
        "SEASON",
    ],
)

season_initial["DATE"] = pd.to_datetime(season_initial["DATE"]).dt.tz_localize(
    "US/Eastern"
)

season_gb = (
    season_initial[season_initial["DATE"] > LAST_FORM_DATA_DATE]
    .groupby(["DATE"])[["SEASON"]]
    .first()
    .reset_index()
    .rename(columns={"DATE": "TIMESTAMP"})
)

season_gb.tail()

,TIMESTAMP,SEASON
94,2025-02-07 17:30:53-05:00,16
95,2025-02-07 18:20:16-05:00,16
96,2025-02-07 18:23:43-05:00,16
97,2025-02-07 18:29:30-05:00,16
98,2025-02-07 18:44:06-05:00,16


In [20]:
new_data_ready = pd.merge(
    cat_df, season_gb, on="TIMESTAMP", how="inner", validate="1:1"
)

# Double check that no records were dropped in the merge
assert new_data_ready.shape[0] == cat_df.shape[0]

In [21]:
migrated_data.tail()

,TIMESTAMP,NEW_SESSION,SUID,MAP,PLAYERS,PLAYERS_1ST,PLAYERS_2ND,PLAYERS_3RD,PLAYERS_4TH,CHARACTERS_1ST,CHARACTERS_2ND,CHARACTERS_3RD,CHARACTERS_4TH,SEASON
2647,2025-01-27 21:52:54-05:00,NO,221,Kalimari Desert,4,Domingo,Blake,Regan,Sudur,Toad,Luigi,Mario,Peach,15
2648,2025-01-27 23:40:20-05:00,NO,221,Toad's Turnpike,3,Konnor,Regan,Domingo,NaN,Peach,Yoshi,Toad,NaN,15
2649,2025-01-28 19:40:36-05:00,YES,222,Sherbet Land,4,Cooper,Connor,Blake,Cole,Toad,Peach,Yoshi,Bowser,15
2650,2025-01-28 20:22:58-05:00,NO,222,Yoshi Valley,4,Cole,Connor,Blake,Matt,Peach,Toad,Yoshi,Luigi,15
2651,2025-01-28 22:58:15-05:00,NO,222,Royal Raceway,4,Blake,Cole,Connor,Matt,Peach,Bowser,Yoshi,Toad,15


In [22]:
new_data_ready.head()

,TIMESTAMP,NEW_SESSION,MAP,PLAYERS,PLAYERS_1ST,PLAYERS_2ND,PLAYERS_3RD,PLAYERS_4TH,CHARACTERS_1ST,CHARACTERS_2ND,CHARACTERS_3RD,CHARACTERS_4TH,SEASON
0,2025-01-29 19:13:49-05:00,YES,D.K.'s Jungle,3,Cooper,Blake,Matt,NaN,Toad,Yoshi,Peach,NaN,15
1,2025-01-29 19:13:58-05:00,NO,Koopa Troopa Beach,3,Matt,Blake,Cooper,NaN,Yoshi,Toad,Peach,NaN,15
2,2025-01-29 19:50:03-05:00,NO,Wario Stadium,4,Cole,Blake,Matt,Cooper,Toad,Peach,Yoshi,Bowser,15
3,2025-01-29 20:33:08-05:00,NO,Wario Stadium,4,Cole,Connor,Blake,Domingo,Luigi,Yoshi,Peach,Toad,15
4,2025-01-30 02:03:51-05:00,NO,Sherbet Land,3,Regan,Connor,Domingo,NaN,Toad,Peach,Yoshi,NaN,15


In [23]:
data_concat = (
    pd.concat([migrated_data, new_data_ready])
    .sort_values(by="TIMESTAMP")
    .reset_index(drop=True)
)

data_concat

,TIMESTAMP,NEW_SESSION,SUID,MAP,PLAYERS,PLAYERS_1ST,PLAYERS_2ND,PLAYERS_3RD,PLAYERS_4TH,CHARACTERS_1ST,CHARACTERS_2ND,CHARACTERS_3RD,CHARACTERS_4TH,SEASON
0,2024-06-21 08:00:00-04:00,YES,1.0,Wario Stadium,4,Blake,Cooper,Matt,Garrett,Toad,Bowser,Yoshi,Peach,11
1,2024-06-21 08:05:00-04:00,NO,1.0,D.K.'s Jungle,4,Cooper,Blake,Matt,Domingo,Mario,Peach,Yoshi,Toad,11
2,2024-06-21 08:10:00-04:00,NO,1.0,Sherbet Land,4,Cooper,Blake,Colton,Garrett,Yoshi,Toad,Bowser,Peach,11
3,2024-06-21 08:15:00-04:00,NO,1.0,Koopa Troopa Beach,4,Blake,Cooper,Matt,Cole,Yoshi,Toad,Luigi,Peach,11
4,2024-06-21 08:20:00-04:00,NO,1.0,Yoshi Valley,4,Cooper,Blake,Matt,Cole,Toad,Peach,Luigi,Yoshi,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,2025-02-07 17:30:53-05:00,YES,NaN,Yoshi Valley,4,Cooper,Cole,Blake,Garrett,Luigi,Toad,Yoshi,Peach,16
2747,2025-02-07 18:20:16-05:00,NO,NaN,Sherbet Land,4,Blake,Cole,Cooper,Garrett,Luigi,Toad,Yoshi,Peach,16
2748,2025-02-07 18:23:43-05:00,NO,NaN,D.K.'s Jungle,4,Blake,Cole,Cooper,Garrett,Yoshi,Toad,Mario,Peach,16
2749,2025-02-07 18:29:30-05:00,NO,NaN,Koopa Troopa Beach,4,Blake,Regan,Cooper,Cole,Peach,Luigi,Yoshi,Toad,16


In [24]:
assert data_concat["TIMESTAMP"].is_unique
assert data_concat["TIMESTAMP"].is_monotonic_increasing

In [25]:
data_concat_filled = fill_new_session(
    df=data_concat, timestamp_column_name="TIMESTAMP", drop_window_start_column=True
)

# # Convert back to EST for Postgres
# data_concat_filled["TIMESTAMP"] = data_concat_filled["TIMESTAMP"].dt.tz_convert(
#     "US/Eastern"
# )

data_concat_filled

,TIMESTAMP,NEW_SESSION,SUID,MAP,PLAYERS,PLAYERS_1ST,PLAYERS_2ND,PLAYERS_3RD,PLAYERS_4TH,CHARACTERS_1ST,CHARACTERS_2ND,CHARACTERS_3RD,CHARACTERS_4TH,SEASON
0,2024-06-21 08:00:00-04:00,YES,1.0,Wario Stadium,4,Blake,Cooper,Matt,Garrett,Toad,Bowser,Yoshi,Peach,11
1,2024-06-21 08:05:00-04:00,NO,1.0,D.K.'s Jungle,4,Cooper,Blake,Matt,Domingo,Mario,Peach,Yoshi,Toad,11
2,2024-06-21 08:10:00-04:00,NO,1.0,Sherbet Land,4,Cooper,Blake,Colton,Garrett,Yoshi,Toad,Bowser,Peach,11
3,2024-06-21 08:15:00-04:00,NO,1.0,Koopa Troopa Beach,4,Blake,Cooper,Matt,Cole,Yoshi,Toad,Luigi,Peach,11
4,2024-06-21 08:20:00-04:00,NO,1.0,Yoshi Valley,4,Cooper,Blake,Matt,Cole,Toad,Peach,Luigi,Yoshi,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,2025-02-07 17:30:53-05:00,YES,NaN,Yoshi Valley,4,Cooper,Cole,Blake,Garrett,Luigi,Toad,Yoshi,Peach,16
2747,2025-02-07 18:20:16-05:00,NO,NaN,Sherbet Land,4,Blake,Cole,Cooper,Garrett,Luigi,Toad,Yoshi,Peach,16
2748,2025-02-07 18:23:43-05:00,NO,NaN,D.K.'s Jungle,4,Blake,Cole,Cooper,Garrett,Yoshi,Toad,Mario,Peach,16
2749,2025-02-07 18:29:30-05:00,NO,NaN,Koopa Troopa Beach,4,Blake,Regan,Cooper,Cole,Peach,Luigi,Yoshi,Toad,16


In [26]:
new_session_df = data_concat_filled.copy()

new_session_df["temp_session"] = np.where(new_session_df["NEW_SESSION"] == "YES", 1, 0)

new_session_df["SUID"] = new_session_df["temp_session"].cumsum()

In [27]:
out_df = new_session_df[
    [
        "TIMESTAMP",
        "NEW_SESSION",
        "SUID",
        "MAP",
        "PLAYERS",
        "PLAYERS_1ST",
        "PLAYERS_2ND",
        "PLAYERS_3RD",
        "PLAYERS_4TH",
        "CHARACTERS_1ST",
        "CHARACTERS_2ND",
        "CHARACTERS_3RD",
        "CHARACTERS_4TH",
        "SEASON",
    ]
].copy()

out_df

,TIMESTAMP,NEW_SESSION,SUID,MAP,PLAYERS,PLAYERS_1ST,PLAYERS_2ND,PLAYERS_3RD,PLAYERS_4TH,CHARACTERS_1ST,CHARACTERS_2ND,CHARACTERS_3RD,CHARACTERS_4TH,SEASON
0,2024-06-21 08:00:00-04:00,YES,1,Wario Stadium,4,Blake,Cooper,Matt,Garrett,Toad,Bowser,Yoshi,Peach,11
1,2024-06-21 08:05:00-04:00,NO,1,D.K.'s Jungle,4,Cooper,Blake,Matt,Domingo,Mario,Peach,Yoshi,Toad,11
2,2024-06-21 08:10:00-04:00,NO,1,Sherbet Land,4,Cooper,Blake,Colton,Garrett,Yoshi,Toad,Bowser,Peach,11
3,2024-06-21 08:15:00-04:00,NO,1,Koopa Troopa Beach,4,Blake,Cooper,Matt,Cole,Yoshi,Toad,Luigi,Peach,11
4,2024-06-21 08:20:00-04:00,NO,1,Yoshi Valley,4,Cooper,Blake,Matt,Cole,Toad,Peach,Luigi,Yoshi,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2746,2025-02-07 17:30:53-05:00,YES,232,Yoshi Valley,4,Cooper,Cole,Blake,Garrett,Luigi,Toad,Yoshi,Peach,16
2747,2025-02-07 18:20:16-05:00,NO,232,Sherbet Land,4,Blake,Cole,Cooper,Garrett,Luigi,Toad,Yoshi,Peach,16
2748,2025-02-07 18:23:43-05:00,NO,232,D.K.'s Jungle,4,Blake,Cole,Cooper,Garrett,Yoshi,Toad,Mario,Peach,16
2749,2025-02-07 18:29:30-05:00,NO,232,Koopa Troopa Beach,4,Blake,Regan,Cooper,Cole,Peach,Luigi,Yoshi,Toad,16


In [28]:
new_records = out_df[out_df["TIMESTAMP"].isin(new_df["Timestamp"])].reset_index(drop=True)

new_records

,TIMESTAMP,NEW_SESSION,SUID,MAP,PLAYERS,PLAYERS_1ST,PLAYERS_2ND,PLAYERS_3RD,PLAYERS_4TH,CHARACTERS_1ST,CHARACTERS_2ND,CHARACTERS_3RD,CHARACTERS_4TH,SEASON
0,2025-01-29 19:13:49-05:00,YES,223,D.K.'s Jungle,3,Cooper,Blake,Matt,NaN,Toad,Yoshi,Peach,NaN,15
1,2025-01-29 19:13:58-05:00,NO,223,Koopa Troopa Beach,3,Matt,Blake,Cooper,NaN,Yoshi,Toad,Peach,NaN,15
2,2025-01-29 19:50:03-05:00,NO,223,Wario Stadium,4,Cole,Blake,Matt,Cooper,Toad,Peach,Yoshi,Bowser,15
3,2025-01-29 20:33:08-05:00,NO,223,Wario Stadium,4,Cole,Connor,Blake,Domingo,Luigi,Yoshi,Peach,Toad,15
4,2025-01-30 02:03:51-05:00,NO,223,Sherbet Land,3,Regan,Connor,Domingo,NaN,Toad,Peach,Yoshi,NaN,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2025-02-07 17:30:53-05:00,YES,232,Yoshi Valley,4,Cooper,Cole,Blake,Garrett,Luigi,Toad,Yoshi,Peach,16
95,2025-02-07 18:20:16-05:00,NO,232,Sherbet Land,4,Blake,Cole,Cooper,Garrett,Luigi,Toad,Yoshi,Peach,16
96,2025-02-07 18:23:43-05:00,NO,232,D.K.'s Jungle,4,Blake,Cole,Cooper,Garrett,Yoshi,Toad,Mario,Peach,16
97,2025-02-07 18:29:30-05:00,NO,232,Koopa Troopa Beach,4,Blake,Regan,Cooper,Cole,Peach,Luigi,Yoshi,Toad,16


In [29]:
out_df.to_csv(
    rf"C:\Users\Cooper\sandbox\mkstream\form_data_migration\form_data_valid.csv",
    index=False,
)

new_records.to_csv(
    rf"C:\Users\Cooper\sandbox\mkstream\form_data_migration\form_data_valid_new_records.csv",
    index=False,
)